In [4]:
from openai import OpenAI
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Filter, FieldCondition, MatchValue, Distance, VectorParams

client = OpenAI(
    api_key= "sk-proj-6obtcJjcFARybp6UqnWPT3BlbkFJ7S3eTF5gaVCpXZfDtaTv",
    organization = "org-5wZe424Sz9nIIZ1Nxgf7n2cv"
)

In [5]:
def vectorize_text(text):

    embeddings = client.embeddings.create(
        input=text,
        model="text-embedding-3-small",
        encoding_format="float"
    )

    return np.array(embeddings.data[0].embedding)

In [ ]:
my_text = "A cat has four legs"
vectorize_text(my_text).shape

In [8]:
qc = QdrantClient(
    url="https://dafd8b74-38ea-418d-88ca-adf6dcfd77a0.us-east4-0.gcp.cloud.qdrant.io",
    port=6333,
    api_key="FyCZ6JSpMVzV0LwEgPF5UMbslatWLRe7SNwf8lIh3dX500BEh7yX_w"
)

In [10]:
qc.create_collection(
    collection_name="testtest2",
    vectors_config=VectorParams(
        # important to know, can find shape of the embeddings of the particular model we used
        size=1536,
        distance=Distance.COSINE
    )
)

True

In [12]:
point1 = PointStruct(
    id=1,
    vector=vectorize_text("A cat has four legs"),
    payload={
        "doc" : "A cat has four legs",
        "pagenumber" : "blabla",
        "url" : "hello.world.ai"
    }
)

point2 = PointStruct(
    id=2,
    vector=vectorize_text("An octopus has eight legs"),
    payload={
        "doc" : "A dog has four legs",
        "pagenumber" : "blabla",
        "url" : "hello.world.ai"
    }
)

point3 = PointStruct(
    id=3,
    vector=vectorize_text("A human has two legs"),
    payload={
        "doc" : "A human has two legs",
        "pagenumber" : "blabla",
        "url" : "hello.world.ai"
    }
)

# adding the vectors to the vectorDB - Qdrant
qc.upsert(collection_name="testtest2", points=[point1, point2, point3])

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
query_vector =  vectorize_text("What has the most number of legs?")
hits = qc.search(
    collection_name="testtest2",
    query_vector=query_vector,
    limit=1
)

In [18]:
nearest_result = hits[0]
print(nearest_result)

id=3 version=0 score=0.6393445 payload={'doc': 'Josey is a durian', 'pagenumber': 'blabla', 'url': 'hello.world.ai'} vector=None shard_key=None
